In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

import sqlite3
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

The code below was provided by Abhineet

In [2]:
from glob import glob

In [3]:
csv_files = glob("./zippedData/*.csv.gz")

In [4]:
csv_files_dict = {}
for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_") # cleaning the filenames
    filename_df = pd.read_csv(filename, index_col=0)
    csv_files_dict[filename_cleaned] = filename_df

Below, we start our own code

Since we will be working with Pandas we need to organize our DFs. 
Let's start by renaming all of them and adding them to a new dictionary. 
By creating a new dictionary of DFs, we can manipulate the data without messing up the originals in csv_files_dict.

In [5]:
df_repository = {} #by creating a dictionary, we can manipulate the data without messing up the originals in csv_files_dict
for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_") # cleaning the filenames    
    exec("""df_repository[""" + """'""" + filename_cleaned + """']""" + """ = csv_files_dict['""" + filename_cleaned + """']""" )

As seen below, the dataframes contain duplicates, which should be removed

In [6]:
for db in df_repository.keys():
    print(db, 'contains ', df_repository[db].duplicated().sum(), ' duplicates')

bom_movie_gross_gz contains  11  duplicates
imdb_name_basics_gz contains  239  duplicates
imdb_title_akas_gz contains  2220  duplicates
imdb_title_basics_gz contains  122  duplicates
imdb_title_crew_gz contains  22915  duplicates
imdb_title_principals_gz contains  114187  duplicates
imdb_title_ratings_gz contains  45074  duplicates
tmdb_movies_gz contains  1020  duplicates
tn_movie_budgets_gz contains  0  duplicates


In [7]:
def clean_duplicates(dataframe): #returns a dataframe without duplicates
    cleaned_filter = dataframe.duplicated()
    index_to_drop = dataframe.loc[cleaned_filter].index
    return dataframe.drop(index=index_to_drop)

In [8]:
for db in df_repository.keys(): #loop to parse dataframes through the clean_duplicates() function we wrote above
    df_repository[db] = clean_duplicates(df_repository[db])

In [9]:
for db in df_repository.keys():
    print(db, 'contains ', df_repository[db].duplicated().sum(), ' duplicates') #checking if it worked

bom_movie_gross_gz contains  0  duplicates
imdb_name_basics_gz contains  0  duplicates
imdb_title_akas_gz contains  0  duplicates
imdb_title_basics_gz contains  0  duplicates
imdb_title_crew_gz contains  0  duplicates
imdb_title_principals_gz contains  0  duplicates
imdb_title_ratings_gz contains  0  duplicates
tmdb_movies_gz contains  0  duplicates
tn_movie_budgets_gz contains  0  duplicates


Empty cells and Place holders:



In [10]:
def NaN_to_zero(DataFrame, DataSeries, change_into=0): #we can change into anything, if we adjust the 'change_into' argument
    filt = DataSeries.isna()
    DataFrame.loc[filt] = change_into
    return DataFrame

In [ ]:
#checking where the NaN ares:
for df in df_repository.keys():
    print('----xxxx------')
    print('DF: ', df)
    print(df_repository[df].isna().sum())
    print('----xxxx------')
          

In [19]:
df_repository['tn_movie_budgets_gz'].release_date = pd.to_datetime(df_repository['tn_movie_budgets_gz'].release_date)

In [ ]:
filt_orig_title = (df_repository['imdb_title_akas_gz'].is_original_title == 1)
df_id_title = df_repository['imdb_title_akas_gz'].loc[filt_orig_title][['title']]
df_id_title


In [ ]:
df_id_title.join(df_repository['imdb_title_crew_gz'],how='inner').drop(columns='writers')

In [ ]:
fuzzy_duplicates_list(df_repository['tn_movie_budgets_gz'].movie, df_id_title.reset_index().title_id)

#df_repository['tn_movie_budgets_gz'] #i need to create a index/pivot to join with imbd data

In [ ]:
df_repository['imdb_name_basics_gz'].head()

Now that we know it works, we can start cleaning up NaN values. After looking through the DFs, we should find other numerical columns with NaN values, and then apply the function to them. 

Variable Types:

Some columns that are supposed to be numerical actually have data stored as strings. Let's write a function that transforms the data

In [12]:
def string_to_float(string):
    if string == None:
        new_string = 0
    else:
        if (type(string) != int) and (type(string) != float):
            new_string = string.replace('$', '')
            new_string = new_string.replace(',', '')

        else:
            new_string = string
    return float(new_string)


def series_string_to_float(dataseries):
    new_dataseries = dataseries.apply(string_to_float)
    return new_dataseries


In [13]:
#cleaning data - from string to float (tn_movie_budgets_gz)
df_repository['tn_movie_budgets_gz'].production_budget = series_string_to_float(df_repository['tn_movie_budgets_gz'].production_budget)
df_repository['tn_movie_budgets_gz'].domestic_gross = series_string_to_float(df_repository['tn_movie_budgets_gz'].domestic_gross)
df_repository['tn_movie_budgets_gz'].worldwide_gross = series_string_to_float(df_repository['tn_movie_budgets_gz'].worldwide_gross)
df_repository['tn_movie_budgets_gz'].release_date = pd.to_datetime(df_repository['tn_movie_budgets_gz'].release_date)

In [18]:
#the data in df_bom_movie_gross_gz.foreign_gross is stored as strings instead of numerical values
df_repository['bom_movie_gross_gz'].loc[:,['foreign_gross']] = series_string_to_float(df_repository['bom_movie_gross_gz'].foreign_gross)

We can join DF using the movie title as the key. For such, we need to
    -standardize the column names
    -clean data (eliminate NaN, placeholders)
    -create new keys (is key is a series of strings, we need to account for typos)
    -join the dataframes
    -analyse and plot

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
def fuzzy_duplicates(string, DataSeries, ratio=90):
    fuzzy_matches = []
    for i in DataSeries:
        result = fuzz.ratio(string, i)
        if (result > ratio) and (result != 100):
            fuzzy_matches.append([string, i, result])
    return fuzzy_matches

In [ ]:
def fuzzy_duplicates_list(Series1, Series2):
    fuzzy_duplicates_list = []
    for string in Series1:
        matches = fuzzy_duplicates(string, Series2)
        if len(matches) >0:
            fuzzy_duplicates_list.append(matches)
    return fuzzy_duplicates_list

In [ ]:
def intersect_lists(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

#test_list = intersect_lists(tn_movies, bom_movies)

#df_bom_movie_gross_gz.set_index(drop=False, keys='title')
#df_tn_movie_budgets_gz.rename(columns={'movie': 'title'}, inplace=True)
#df_tn_movie_budgets_gz.set_index(keys='title', inplace=True)


In [ ]:
df_tn_movie_budgets_gz.join(df_bom_movie_gross_gz, how='outer', sort=False, lsuffix='_tn_', rsuffix='_bom_' ).head(50)

After cleaning data, we can start creating new features

In [ ]:
df_bom_movie_gross_gz.foreign_gross = series_string_to_float(df_bom_movie_gross_gz.foreign_gross)

In [ ]:
df_bom_movie_gross_gz['total_gross'] = df_bom_movie_gross_gz['foreign_gross'].add(df_bom_movie_gross_gz['domestic_gross'], fill_value=0.0)
df_bom_movie_gross_gz.head()

In order to simplify the manipulation of data, the column names of each df could be standardized ('title' vs 'movie title' vs 'original title', etc)

In [21]:
df_repository['imdb_title_crew_gz'].groupby('directors').sum()

,writers
directors,
nm0000080,"nm0000080,nm0462648"
nm0000095,nm0000095
nm0000108,"nm0160391,nm0617789,nm0000108nm0292451nm000010..."
nm0000110,"nm0002005,nm0338169nm3092414,nm0462895,nm00020..."
nm0000116,"nm0000116,nm0415425,nm0004307,nm0798646nm00001..."
...,...
nm9992900,"nm9992900,nm9992899"
nm9992905,0
nm9992906,nm9992906


In [22]:
df_repository['imdb_title_crew_gz'].head()

,directors,writers
tconst,,
tt0285252,nm0899854,nm0899854
tt0438973,NaN,"nm0175726,nm1802864"
tt0462036,nm1940585,nm1940585
tt0835418,nm0151540,"nm0310087,nm0841532"
tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


In [45]:
def split_series(series, split_characters):
    broken_series = series.str.split(split_characters)
    new_list1 = []
    new_list2 = []
    new_list3 = []
    
    for lst in broken_series:
        if type(lst) == list:
            new_list1.append(lst[0])
            
            if len(lst) > 1:
                new_list2.append(lst[1])
                
                if len(lst) > 2:
                    new_list3.append(lst[2])
                else:
                    new_list3.append('N/A')
            
            
            else: 
                new_list2.append('N/A')
                new_list3.append('N/A')
            
        else:
            new_list1.append('N/A')
            new_list2.append('N/A')
            new_list3.append('N/A')
            
    new_series_dict = {'dir_code_1': new_list1, 'dir_code_2': new_list2, 'dir_code_3': new_list3 }
    return pd.DataFrame(new_series_dict)

In [46]:
split_series(df_repository['imdb_title_crew_gz'].directors, ',')

,dir_code_1,dir_code_2,dir_code_3
0,nm0899854,N/A,N/A
1,N/A,N/A,N/A
2,nm1940585,N/A,N/A
3,nm0151540,N/A,N/A
4,nm0089502,nm2291498,nm2292011
...,...,...,...
123224,nm10122357,N/A,N/A
123225,nm6711477,N/A,N/A
123226,nm10123242,nm10123248,N/A
123227,nm4993825,N/A,N/A


In [82]:
split_series(df_repository['imdb_title_crew_gz'].directors, ',').dir_code_1.value_counts()

N/A           1163
nm2551464       23
nm2410514       22
nm0482774       19
nm0290556       19
              ... 
nm5363550        1
nm4592235        1
nm8226302        1
nm10723839       1
nm3276178        1
Name: dir_code_1, Length: 94755, dtype: int64

In [61]:
new_series_directors =  df_repository['imdb_name_basics_gz'].known_for_titles.str.split(',')
filt = new_series_directors.isna()
new_series_directors.loc[filt] = '.'

In [80]:
list_ = []

for index, lst in new_series_directors:
    list_.append(len(lst), index)

list_


ValueError: too many values to unpack (expected 2)

In [87]:
#df_repository['imdb_name_basics_gz'].primary_profession.str.split(',')
df_repository['imdb_name_basics_gz'].primary_profession = df_repository['imdb_name_basics_gz'].primary_profession.str.split(',')
df_repository['imdb_name_basics_gz'].known_for_titles = df_repository['imdb_name_basics_gz'].known_for_titles.str.split(',')

In [88]:
df_repository['imdb_name_basics_gz'].head()

,primary_name,birth_year,death_year,primary_profession,known_for_titles
nconst,,,,,
nm0061671,Mary Ellen Bauder,NaN,NaN,"[miscellaneous, production_manager, producer]","[tt0837562, tt2398241, tt0844471, tt0118553]"
nm0061865,Joseph Bauer,NaN,NaN,"[composer, music_department, sound_department]","[tt0896534, tt6791238, tt0287072, tt1682940]"
nm0062070,Bruce Baum,NaN,NaN,"[miscellaneous, actor, writer]","[tt1470654, tt0363631, tt0104030, tt0102898]"
nm0062195,Axel Baumann,NaN,NaN,"[camera_department, cinematographer, art_depar...","[tt0114371, tt2004304, tt1618448, tt1224387]"
nm0062798,Pete Baxter,NaN,NaN,"[production_designer, art_department, set_deco...","[tt0452644, tt0452692, tt3458030, tt2178256]"
